In [3]:
import socket
import sticks

listen_socket = socket.socket()
listen_socket.bind(('127.0.0.1', 6789))
listen_socket.listen()

game_socket, addr = listen_socket.accept()

# create board size
size = int(input("Indicate size of board: "))
game_socket.sendall(b''+str(size).encode()+b'\n')
game = sticks.sticks(size)

while True:
    # print board while game is not won
    game.render_sticks()

    # get input from player
    col = input("Select Column:")
    row = input("Select Row: ")
    amt = input("Select Amount: ")

    while game.remove_sticks(col, row, amt):
        print("Your input is invalid. Please try again.")
        col = input("Select Column: ")         
        row = input("Select Row: ")
        amt = input("Select Amount: ")

    game.remove_sticks(col, row, amt)
    game.render_sticks()

    # if after player turn. left 1 stick, they win
    if game.sticks == 1:
        game_socket.sendall(b'WON' + str(col).encode() + str(row).encode() + str(amt).encode()+ b'\n')
        print("You Won!")
        break
        
    game_socket.sendall(b'MOVE' + str(col).encode() + str(row).encode()+ str(amt).encode()+ b'\n')
    
    #recieving from client
    recieved = b''
    print('Waiting for opponent...')
    while b'\n' not in recieved:
        recieved += game_socket.recv(1024)
    if recieved.startswith(b'END'):
        print("You win, Opponent Quits")
        break
    if recieved.startswith(b'WON'):
        opp_move = recieved.decode()   
        opp_move = opp_move[3:]
        col = str(opp_move[0])
        row = str(opp_move[1])
        amt = str(opp_move[2])
        game.remove_sticks(col,row,amt) 
        game.render_sticks()
        print("Opponent Won!")
        break
    if recieved.startswith(b'MOVE'):
        opp_move = recieved.decode()   
        opp_move = opp_move[4:]
        
    col = str(opp_move[0])
    row = str(opp_move[1])
    amt = str(opp_move[2])
    game.remove_sticks(col,row,amt) 
    
game_socket.close()
listen_socket.close()

Indicate size of board: 3
Remaining Sticks: 6
  0 1 2
0 1
1 1 1
2 1 1 1
Select Column:0
Select Row: 2
Select Amount: 3
Remaining Sticks: 3
  0 1 2
0 1
1 1 1
2 - - -
Waiting for opponent...
Remaining Sticks: 2
  0 1 2
0 -
1 1 1
2 - - -
Select Column:1
Select Row: 1
Select Amount: 1
Remaining Sticks: 1
  0 1 2
0 -
1 1 -
2 - - -
Remaining Sticks: 1
  0 1 2
0 -
1 1 -
2 - - -
You Won!
